<a href="https://colab.research.google.com/github/RomaTruba/dzSEM/blob/main/s_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
import pandas as pd

In [ ]:
# Создаем небольшой датасет для демонстрации
data = {
    "text": [
        "Это просто прекрасный день!",
        "Я обожаю это место.",
        "Ненавижу, когда так происходит.",
        "Ужасное обслуживание в этом кафе.",
        "Отличный фильм, рекомендую!",
        "Разочарован качеством товара.",
        "Прекрасная погода сегодня.",
        "Слишком дорого за такое качество."
    ],
    "label": [1, 1, 0, 0, 1, 0, 1, 0]  # 1 - позитивный, 0 - негативный
}

In [ ]:
# Создаем DataFrame и преобразуем в Dataset
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

In [ ]:
# Разделяем на train и test
dataset = dataset.train_test_split(test_size=0.25, seed=42)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})


In [ ]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Удаляем текстовую колонку, оставляем только токены
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Устанавливаем формат для PyTorch
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-10-530b8abb8317>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Запуск обучения
trainer.train()

# Оценка модели
eval_results = trainer.evaluate()
print(f"Результаты оценки: {eval_results}")

Epoch,Training Loss,Validation Loss
1,No log,0.685588
2,No log,0.675172
3,No log,0.670085


Результаты оценки: {'eval_loss': 0.6700847744941711, 'eval_runtime': 0.8948, 'eval_samples_per_second': 2.235, 'eval_steps_per_second': 1.118, 'epoch': 3.0}


In [ ]:
# Функция для предсказания
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return "Позитивный" if predictions.item() == 1 else "Негативный"

# Тестируем на новых примерах
test_texts = [
    "Мне понравилось это приложение!",
    "Очень плохая погода сегодня.",
    "Среднего качества товар, но дорогой."
]

for text in test_texts:
    print(f"Текст: {text}")
    print(f"Тональность: {predict_sentiment(text)}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Текст: Мне понравилось это приложение!
Тональность: Позитивный

Текст: Очень плохая погода сегодня.
Тональность: Позитивный

Текст: Среднего качества товар, но дорогой.
Тональность: Позитивный

